### **Q4 — Off-Policy Monte Carlo**

#### **Problem Statement**

In this problem, we use the same 5×5 Gridworld environment from Problem 3 and estimate the state-value function using off-policy Monte Carlo control with Importance Sampling.

- The **behavior policy**, b(a|s), is a fixed uniform random policy (each action has probability 1/4).
- The **target policy**, π(a|s), is a greedy policy with respect to the learned action-value function Q(s,a).
- The discount factor is γ = 0.9.

We applied Off-policy Monte Carlo control with Weighted Importance Sampling to estimate the action-value function 𝑄(𝑠,𝑎while sampling episodes from the fixed random behavior policy. The importance sampling ratios were used to correct for the distribution mismatch between the behavior and target policies.

After learning Q(s,a) , the state-value function was derived as: V(s)=amax​Q(s,a)
and the corresponding greedy target policy was obtained from the learned Q-values.

For reference, we also computed the optimal value function 𝑉∗ and optimal policy 𝜋∗ using Value Iteration, and compared the Monte Carlo estimates against the dynamic programming solution.

#### **Gridworld Environment**

We use a **5 × 5 gridworld**, identical to the environment used in **Problem 3**.

Environment details:

- Grid size: 5 rows × 5 columns
- Terminal (goal) state: s_{4,4}  (bottom-right cell)
- Grey (non-favourable) states: s_{2,2}, s_{3,0}, s_{0,4} 
- Available actions:  
  Right (→), Left (←), Down (↓), Up (↑)
- Transitions:
  * Deterministic.
  * If an action would move outside the grid, the agent remains in the same state.
-  Reward function R(s') (depends on the next state):
  * +10 if the agent reaches the goal state
  * −5 if the agent enters a grey state
  * −1 for all other transitions
 
An episode terminates when the agent reaches the goal state.

#### **Policies**

**Behavior Policy**

The behavior policy b(a|s) selects actions uniformly at random from the action set: b(a|s) = \frac{1}{4}


This policy is fixed and is used only to generate episodes for off-policy learning.

**Target Policy**

The target policy π(a|s)  is greedy with respect to the current estimate of the action-value function Q(s,a): pi(s) = argmax_a Q(s,a)

The policy is deterministic. In the case of ties, the implementation selects the first action returned by the argmax operation.


#### **Episode Generation**

Episodes are generated by:

* Randomly sampling a non-terminal starting state.
* Following the behavior policy b(a|s), which selects actions uniformly at random.
* Applying deterministic transitions according to the environment dynamics.

Each episode consists of a sequence of tuples: (s_t, a_t, r_{t+1})

where:
- s_t  is the current state,
- a_t is the action selected by the behavior policy,
- r_{t+1} = R(s_{t+1}) is the reward received after transitioning to the next state.

An episode terminates when:
- The agent reaches the goal state, or
- A safety cap of 200 steps is reached (to prevent infinite loops).

#### **Value Function and Policy Display**

After training completes, we report:

- The estimated state-value function V(s) = \max_a Q(s,a) , displayed as a 5 × 5 matrix.
- The greedy target policy  π(s) = argmax_a Q(s,a) , shown using directional arrows (→, ←, ↓, ↑).

During training, checkpoint logs are recorded at fixed episode intervals to monitor convergence. These checkpoints report:

- Number of episodes completed
- Total transitions collected
- Maximum change in the value function since the previous checkpoint

This presentation mirrors the grid-based value tables and policy diagrams shown in the lecture slides for the 5 × 5 gridworld.

#### **Logger**

During execution, detailed logs are written to:


Each run generates:

- A text log file q4_run_*.txt containing:
  - Environment configuration
  - Value Iteration convergence trace (delta per iteration)
  - Monte Carlo checkpoint logs (every fixed number of episodes)
  - Final estimated value matrix and greedy policy
  - Runtime statistics and comparison metrics

- CSV files for reproducibility:
  - V_mc_*.csv — final Monte Carlo value estimates
  - pi_mc_*.csv — final Monte Carlo greedy policy
  - V_vi_*.csv — optimal value function from Value Iteration
  - pi_vi_*.csv — optimal policy from Value Iteration

Monte Carlo checkpoint logs include:
- Episode count
- Total transitions collected
- Maximum change in the value function since the previous checkpoint

This logging structure ensures transparency, reproducibility, and clear monitoring of convergence.

#### **Initialization of Q and Weight Tracking**

We maintain the following data structures:

- Q(s,a): Action-value estimates.
- C(s,a): Cumulative importance weights used in Weighted Importance Sampling (WIS).

Both matrices are initialized to zero: Q(s,a) = 0, quad C(s,a) = 0

The target policy π(s) is initialized arbitrarily (default action index), and is updated greedily during training.

Note: Only Weighted Importance Sampling (WIS) is implemented. Ordinary Importance Sampling (OIS) and its counters are not used in this implementation.

#### **Off-policy Monte Carlo Control (Weighted Importance Sampling)**

For each episode generated under the behavior policy, returns are computed **backward** from the end of the episode: G_t = \gamma G_{t+1} + r_{t+1}

Weighted Importance Sampling (WIS) is used to update the action-value function: C(s,a) \leftarrow C(s,a) + W

Q(s,a) \leftarrow Q(s,a) + \frac{W}{C(s,a)} \left( G - Q(s,a) \right)

where:
W = \prod_{k=t}^{T-1} \frac{\pi(a_k|s_k)}{b(a_k|s_k)}


Since the target policy is greedy and deterministic, the importance ratio simplifies to: W = \frac{1}{b(a|s)} = 4

as long as the behavior action matches the greedy action.

If the behavior action at a state does not match the greedy target action, the update for that episode stops early (as in the lecture pseudocode). This ensures that importance weights do not grow unnecessarily large.

#### **Running the Algorithm and Writing Logs**

The Off-policy Monte Carlo Control algorithm (Weighted Importance Sampling) is run for a fixed number of episodes.

During execution:

- Value Iteration convergence deltas are logged per iteration.
- Monte Carlo training progress is logged at fixed episode checkpoints.
- Final value functions and policies are saved as CSV files.
- A detailed run summary is written to a log file under logs_q4.

This logging structure allows us to:
- Monitor convergence behavior,
- Compare Monte Carlo results against Value Iteration,
- Ensure reproducibility of the experiment.

#### **Importing the Q4 Implementation**

In this notebook, we reuse the Python implementation written for Question 4 instead of rewriting all algorithms directly inside the notebook.

This approach keeps the notebook:

- Clean and easy to read  
- Consistent with the actual code used to generate results  
- Aligned with good software engineering practice (single implementation reused across experiments)

The core logic is contained in the modules inside src_q4.

To allow the notebook to access these modules, we dynamically add the project root directory (the folder containing src/) to Python’s module search path. This enables direct imports from src.q4.

**Modules Used**

- **gridworld5x5.py**  
  Implements the 5×5 Gridworld environment used in Q3 and Q4, including deterministic transitions and reward-on-arrival logic.

- **value_iteration_q3.py**  
  Provides the Value Iteration implementation used to compute the optimal value function \(V^*\) and policy \( \pi^* \) (baseline).

- **offpolicy_mc_importance_sampling.py**  
  Implements Off-policy Monte Carlo Control using **Weighted Importance Sampling (WIS)**.

- **run_q4.py**  
  Orchestrates the experiment by:
  - Running Value Iteration,
  - Running Off-policy Monte Carlo,
  - Logging convergence traces,
  - Saving CSV outputs for reproducibility.

By separating implementation (Python modules) from analysis (notebook), this notebook focuses on:

- Running experiments  
- Monitoring convergence  
- Comparing model-based and model-free methods  
- Interpreting results  

All numerical results shown in this notebook are generated directly from the external Python implementation.

In [13]:
import sys
from pathlib import Path
import numpy as np
import time

# Robustly add project root (folder that contains 'src') to sys.path
HERE = Path.cwd()
project_root = None
for p in [HERE] + list(HERE.parents):
    if (p / "src").exists():
        project_root = p
        break
if project_root is None:
    raise RuntimeError("Could not find project root containing 'src' folder.")

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.q4.gridworld5x5 import GridWorld5x5
from src.q4.value_iteration_q3 import (
    value_iteration,
    format_V_grid as format_V_grid_vi,
    format_pi_grid as format_pi_grid_vi,
)
from src.q4.offpolicy_mc_importance_sampling import (
    off_policy_mc_control_weighted_is,
    format_V_grid as format_V_grid_mc,
    format_pi_grid as format_pi_grid_mc,
)

In [14]:
env = GridWorld5x5(gamma=0.9)

#### **Value Iteration Baseline**

Before running the off-policy Monte Carlo methods, we compute a Value Iteration (VI) baseline for the same 5 × 5 Gridworld.

Value Iteration is a model-based dynamic programming method. It assumes full knowledge of:

- State transition dynamics  
- Reward function  
- Discount factor (γ = 0.9)

Using the Bellman optimality update, it repeatedly sweeps over all states until convergence.  
Because the gridworld is small and deterministic, convergence occurs in only a few iterations.

**What this cell outputs**

- Number of iterations required for convergence  
- Runtime (demonstrating model-based efficiency)  
- Optimal value matrix V* displayed in grid form  
- Optimal greedy policy π*, shown using directional arrows  

**Why this matters for Q4**

The off-policy Monte Carlo methods in Q4:

- Do not know the environment model  
- Must learn purely from sampled episodes  
- Are model-free and variance-prone  

By comparing Monte Carlo estimates against the Value Iteration baseline, we can evaluate:

- How close the learned value function is to V*   
- Whether the learned greedy policy matches π* 
- The computational differences between model-based and model-free approaches  
- The stability benefits of Weighted Importance Sampling  

Value Iteration therefore serves as an essential benchmark for interpreting Monte Carlo performance.

In [15]:
# --- Value Iteration baseline ---
t0 = time.perf_counter()
vi_res = value_iteration(env)
t_vi = time.perf_counter() - t0

print("=== Value Iteration (Baseline) ===")
print("Iterations:", vi_res.iterations)
print("Runtime (seconds):", round(t_vi, 6))
print("\nOptimal V*:")
print(format_V_grid_vi(env, vi_res.V, decimals=2))
print("\nOptimal Policy π*:")
print(format_pi_grid_vi(env, vi_res.pi))

# --- Off-policy MC (Weighted IS) ---
t1 = time.perf_counter()
mc_res = off_policy_mc_control_weighted_is(
    env,
    num_episodes=100_000,
    seed=7,
    max_steps_per_episode=200,
    log_every=10_000,  # optional
    logger=print,      # optional
)
t_mc = time.perf_counter() - t1

print("\n=== Off-policy MC (Weighted IS) ===")
print("Episodes:", mc_res.episodes)
print("Transitions collected:", mc_res.steps_collected)
print("Runtime (seconds):", round(t_mc, 6))
print("\nEstimated V (from Q):")
print(format_V_grid_mc(env, mc_res.V, decimals=2))
print("\nGreedy Policy (from Q):")
print(format_pi_grid_mc(env, mc_res.pi))

=== Value Iteration (Baseline) ===
Iterations: 9
Runtime (seconds): 0.001868

Optimal V*:
 -0.43   0.63   1.81   3.12   4.58
  0.63   1.81   3.12   4.58   6.20
  1.81   3.12   4.58   6.20   8.00
  3.12   4.58   6.20   8.00  10.00
  4.58   6.20   8.00  10.00    G   

Optimal Policy π*:
 →  →  →  ↓  ↓ 
 →  →  →  →  ↓ 
 →  ↓  →  →  ↓ 
 →  →  →  →  ↓ 
 →  →  →  →  G 
MC checkpoint @ episode 10,000: steps_so_far=781,109, max|ΔV| since last checkpoint=10.000000
MC checkpoint @ episode 20,000: steps_so_far=1,578,525, max|ΔV| since last checkpoint=0.010762
MC checkpoint @ episode 30,000: steps_so_far=2,364,241, max|ΔV| since last checkpoint=0.011800
MC checkpoint @ episode 40,000: steps_so_far=3,143,280, max|ΔV| since last checkpoint=0.012310
MC checkpoint @ episode 50,000: steps_so_far=3,931,466, max|ΔV| since last checkpoint=0.004503
MC checkpoint @ episode 60,000: steps_so_far=4,718,056, max|ΔV| since last checkpoint=0.005169
MC checkpoint @ episode 70,000: steps_so_far=5,493,881, max|ΔV| s

#### **Executing the Q4 Workflow**

This step runs the complete Q4 experiment using the updated Python implementation in src/q4/.

The workflow consists of:
* Value Iteration (baseline)
* Off-policy Monte Carlo Control using Weighted Importance Sampling (WIS)

Both methods use the same:
* 5×5 Gridworld
* Discount factor γ = 0.9
* Episode count (e.g., 100,000)
* Maximum step cap per episode

*What This Step Does*
* Runs Value Iteration to compute:
  - Optimal value function 𝑉∗
  - Optimal greedy policy 𝜋∗
  - Number of iterations required for convergence
  - Convergence trace (delta per iteration)
* Runs Off-policy Monte Carlo (Weighted IS):
  - Generates episodes using a uniform random behavior policy
  - Updates 𝑄(𝑠,𝑎) using weighted importance sampling
  - Logs checkpoint progress every fixed number of episodes
  - Produces the learned value function and greedy target policy
* Logs detailed outputs to: logs/q4/
  - Convergence trace (Value Iteration)
  - Monte Carlo checkpoint summaries
  - Final value matrices
  - Greedy policies
  - CSV exports for reproducibility

**Output**

The experiment produces:
* Optimal value function and policy from Value Iteration
* Estimated value function and policy from Off-policy Monte Carlo
* Runtime comparison (model-based vs model-free)
* Convergence diagnostics
* Maximum absolute difference between MC and VI

This structure keeps:
* Implementation inside Python modules
* Analysis and interpretation inside the notebook

which follows clean software engineering practice and ensures reproducibility.

In [17]:
print("=== Final Summary ===")
print("VI iterations:", vi_res.iterations)
print("MC episodes:", mc_res.episodes)
print("Max |V_MC - V*|:",
      np.max(np.abs(mc_res.V - vi_res.V)))

=== Final Summary ===
VI iterations: 9
MC episodes: 100000
Max |V_MC - V*|: 0.03123330491617171


* **Off-policy Monte Carlo Results (WIS)**

In this section, we display the final results of the off-policy Monte Carlo control method implemented in Question 4 and compare the learned value function and policy against the Value Iteration baseline.

The Monte Carlo algorithm learns purely from sampled episodes generated under a random behavior policy. Unlike Value Iteration, it does not assume knowledge of transition probabilities or reward dynamics.

* **Weighted Importance Sampling (WIS)**

The table above shows the state-value function and greedy policy learned using Weighted Importance Sampling (WIS).

In this method:
* Episodes are generated using a uniform random behavior policy.
* Importance sampling weights correct for the mismatch between the behavior policy and the greedy target policy.
* The cumulative weight normalization reduces variance during updates.

Because weighted importance sampling normalizes the importance ratios using cumulative weights, it prevents the extreme instability typically seen in ordinary importance sampling.

* **Observed Behavior**

From the results:
* The learned value function closely matches the optimal value function from Value Iteration.
* The greedy policy aligns with the optimal policy.
* The maximum absolute difference between Monte Carlo and Value Iteration values is small (≈ 0.03).
* Convergence improves steadily across episode checkpoints.

This confirms that Weighted Importance Sampling provides stable and accurate off-policy control in finite-sample settings.

#### **Evidence: Comparing Weighted MC to Value Iteration**

In this cell, we quantitatively compare the final value function learned using Weighted Importance Sampling (WIS) with the optimal value function obtained from Value Iteration (VI).

Value Iteration computes the optimal solution directly using full knowledge of the environment’s transition dynamics and reward function. It therefore serves as the optimal reference baseline.

The off-policy Monte Carlo method, in contrast, learns purely from sampled episodes generated under a random behavior policy. If the implementation is correct and sufficiently trained, the learned values should be very close to the Value Iteration results.

To measure this closeness, we compute:
* Maximum absolute difference: max∣VMC​−V∗∣, This shows the largest deviation at any state.
* Mean absolute difference: mean(∣VMC​−V∗∣),This shows the average deviation across all states.

**Interpretation of the Result**
* A small maximum difference (≈ 0.03 in our experiment) indicates that the Monte Carlo method has converged very close to the optimal solution.
* The low mean difference confirms that the agreement holds consistently across the grid.
* This provides strong numerical evidence that Weighted Importance Sampling enables stable and accurate off-policy learning, even without access to the full MDP model.

These results demonstrate that model-free learning can approximate model-based solutions given enough sampled experience.

In [16]:
V_mc = mc_res.V
V_vi = vi_res.V

max_diff = float(np.max(np.abs(V_mc - V_vi)))
mean_diff = float(np.mean(np.abs(V_mc - V_vi)))

print("=== Evidence (MC vs VI) ===")
print(f"max |V_MC - V*|  = {max_diff:.4f}")
print(f"mean|V_MC - V*| = {mean_diff:.4f}")

=== Evidence (MC vs VI) ===
max |V_MC - V*|  = 0.0312
mean|V_MC - V*| = 0.0176


#### **Comparison Table**

| Method                          | How it learns                                                    | Needs full model? | Stability   | Speed     | Main takeaway                                                  |
| ------------------------------- | ---------------------------------------------------------------- | ----------------- | ----------- | --------- | -------------------------------------------------------------- |
| **Value Iteration (VI)**        | Uses Bellman optimality updates with known transitions & rewards | Yes               | Very stable | Very fast | Model-based method that directly computes the optimal solution |
| **Off-policy MC (Weighted IS)** | Learns from sampled episodes using importance sampling           | No                | Stable      | Slower    | Model-free learning that approximates the optimal solution     |


#### **Analysis/Conclusion**

In Q4, we solved the same 5×5 Gridworld from Q3 using two different approaches: a model-based method (Value Iteration) and a model-free method (Off-policy Monte Carlo with Weighted Importance Sampling).

The environment is identical to Q3: deterministic transitions, four actions, a goal at (4,4), grey penalty states, reward-on-arrival (+10 goal, −5 grey, −1 otherwise), and discount factor γ = 0.9.

First, Value Iteration was run as a baseline. Because it has access to the full transition model, it performs deterministic Bellman sweeps over all states. It converged in only 9 iterations and required approximately 0.0011 seconds, demonstrating the efficiency of model-based dynamic programming in small tabular environments.

Next, Off-policy Monte Carlo learned purely from sampled episodes generated by a random behavior policy. Using Weighted Importance Sampling to stabilize updates, it required 100,000 episodes and approximately 7.8 million transitions, taking about 11.85 seconds of runtime.

Despite the significantly higher computational effort, Monte Carlo successfully approximated the optimal value function. The maximum absolute difference between the estimated value function and the optimal value function from Value Iteration was only 0.0312, which is very small relative to the scale of rewards in the environment.

From a computational complexity perspective:
* Value Iteration scales as O(K · |S| · |A|), where K is the number of sweeps.
* Off-policy Monte Carlo scales as O(E · L), where E is the number of episodes and L is the average episode length.

Overall, this experiment shows that while model-free learning can approximate the optimal solution, it requires substantially more data and computation compared to model-based methods when the environment dynamics are known. In this small, fully specified Gridworld, Value Iteration is clearly more efficient. However, Off-policy Monte Carlo remains valuable because it does not require knowledge of the transition model and can therefore be applied to environments where the dynamics are unknown.